## Setup

In [ ]:
!pip install numpy
!pip install pandas
!pip install scipy
!pip install hmmlearn
!pip install statsmodels

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from hmmlearn import hmm
import gzip
import os
from tqdm import tqdm, trange
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import fisher_exact
import math
import joblib
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# Specify project directories in Sherlock
data_path = '/oak/stanford/groups/mrivas/projects/wgs-constraint-llm/data/'
results_path = '/oak/stanford/groups/mrivas/projects/wgs-constraint-llm/osthoag/wgs-constraint-llm/results/'

# Specify the file paths
variants_file_path = data_path + 'aou_variants.tsv.gz'
gene_annotation_file_path = data_path + 'gencode.v44.basic.annotation.gtf.gz'
genome_coverage_file_path = data_path + 'gnomad.genomes.r3.0.1.coverage.summary.tsv.bgz'
coverage_file_path = data_path + 'gnomad.exomes.v4.0.coverage.summary.tsv.bgz'

## Define helper methods

In [3]:
# Function to extract the observation sequence for the HMM
def get_sequence(chromnum):
    # Filter gene df for given chromosome
    chr_gene_df = gene_df[gene_df['chr'] == 'chr' + str(chromnum)]
    
    # Filter variant df for given chromosome
    chr_variants_df = variants_df[variants_df['chr'] == chromnum]
    
    # Filter coverage data for chromosome 2 based on the coverage threshold
    chr_coverage_df = coverage_df[(coverage_df['chr'] == 'chr' + str(chromnum))]
    
    # Determine the length of the entire sequence based on gene, coverage, and variant data
    sequence_length = max(chr_gene_df['end'].max(), chr_coverage_df['pos'].max(), chr_variants_df['pos'].max())

    # Initialize a boolean mask for protein-coding regions
    protein_coding_mask = np.zeros(sequence_length + 1, dtype=bool)

    # Mark positions that are within protein-coding regions
    for start, end in zip(chr_gene_df['start'].to_numpy(), chr_gene_df['end'].to_numpy()):
        protein_coding_mask[start:end+1] = 1

    # Initialize a boolean mask for regions with sufficient exome coverage
    coverage_mask = np.zeros(sequence_length + 1, dtype=bool)

    # Mark positions with coverage greater than 80%
    coverage_mask[chr_coverage_df['pos'].to_numpy()] = 1

    # Initialize the sequence array (observations) to zeros
    sequence = np.zeros(sequence_length + 1)

    # Mark positions where variants are observed
    sequence[chr_variants_df['pos'].to_numpy()] = 1

    # Filter the sequence for protein-coding regions with sufficient coverage
    observations = np.array(sequence[protein_coding_mask & coverage_mask])
    
    # Identify the positions that correspond to these observations
    positions = np.where(protein_coding_mask & coverage_mask)[0]
    
    return observations, positions

# Function to fit an HMM to the observations
def fit_HMM(observations, order=2):
    # Prepare the observations for HMM fitting by creating higher-order observations
    X = np.stack([observations[i:i-order] for i in range(order)], axis=1)

    # Convert observations to counts
    X_counts = np.column_stack([(X == i).sum(axis=1) for i in range(2)])

    # Instantiate and fit a first-order HMM
    model = hmm.MultinomialHMM(n_components=2, random_state=10)
    model.fit(X_counts)

    return model

# Function to predict probabilities using a fitted HMM
def get_HMM_predictions(observations, model, order=2):
    # Create higher-order observations (context of order-1)
    X = np.stack([observations[i:i-order] for i in range(order)], axis=1)

    # Convert observations to counts for the HMM
    X_counts = np.column_stack([(X == i).sum(axis=1) for i in range(2)])

    # Predict probabilities for each position in the sequence
    probabilities = model.predict_proba(X_counts)

    return probabilities

# Function to perform Ordinary Least Squares (OLS) regression on prediction data
def ols_regression(predictions_df):
    # Add a constant term to the predictor variable
    X = sm.add_constant(predictions_df['prob_0'])

    # Fit the OLS model and print the summary
    model4 = sm.OLS(predictions_df['observation'],X).fit()

    model4.summary().tables[1].pvalues_precision = 100  # Adjust the number of significant digits

    # Print the regression summary
    print(model4.summary())

    # Extract the F-statistic and associated p-value
    f_statistic = model4.fvalue
    p_value_f_statistic = model4.f_pvalue
    
    return f_statistic, p_value_f_statistic

# Function to plot histograms of observed variants and HMM predictions
def plot_hist_from_predictions(predictions_df):
    # Create a figure with two subplots in one row and two columns
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    # Plot the first histogram in the first subplot
    axs[0].hist(predictions_df['observation'], edgecolor="blue")
    axs[0].set_title('Histogram of Observations')
    axs[0].set_xlabel('Has a variant')
    axs[0].set_ylabel('Frequency')

    # Plot the second histogram in the second subplot
    axs[1].hist(predictions_df['prob_0'], edgecolor="blue")
    axs[1].set_title('Histogram of Predictions')
    axs[1].set_xlabel('Probability of 0')
    axs[1].set_ylabel('Frequency')

    # Adjust layout to prevent clipping of titles
    plt.tight_layout()

    # Show the plots
    plt.show()
    
# Function to plot observed mutations for a subsequence
def plot_subsequence_observations(observations, start_idx, end_idx, name='Target Region'): 
    plt.bar(range(start_idx, end_idx), observations[start_idx:end_idx], color='black', width=1.0)#, label='Original Sequence')
    plt.title("Observed mutations for part of " + name)
    plt.xlabel('Position on Chromosome 2')
    plt.yticks([0,1]) 
    plt.ylabel('Observation')
#     plt.legend()
    plt.margins(x=0)
    plt.tight_layout(rect=[0, 0, 2, 0.5])
    plt.show()
    
# Function to plot HMM predictions for a subsequence
def plot_subsequence_predictions(predictions, start_idx, end_idx, name='Target Region'):    
    plt.bar(range(start_idx, end_idx), predictions[start_idx:end_idx, 0], color='#8C1515', width=1.0, label='Probability of 0')
#     plt.bar(range(start_idx, end_idx), predictions[start_idx:end_idx, 1], bottom=predictions[start_idx:end_idx, 0], color='#D2C295', label='Probability of 1')
    plt.title("HMM Predictions for part of " + name)
    plt.xlabel('Position on Chromosome 2')
    plt.ylabel('Probability of 0')
#     plt.legend()
    plt.margins(x=0)
    plt.tight_layout(rect=[0, 0, 2, 0.5])
    plt.show()
    
# Function to plot both observed mutations and HMM predictions for a subsequence
def plot_subsequence(observations, predictions, start_idx, end_idx, gene_name='Target Region', save_name=None):
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 4), sharex=True)
    plt.suptitle("Observed Mutations vs HMM Predictions for " + gene_name, y=0.9)

    # Plot the original sequence
    axes[0].bar(range(start_idx, end_idx), observations[start_idx:end_idx], width=1.0, color='black', label='Original Sequence')
    axes[0].set_ylabel('Observation')
#     axes[0].legend()
    axes[0].margins(x=0)
    
    # Plot the predicted probabilities as stacked barplots
    axes[1].bar(range(start_idx, end_idx), predictions[start_idx:end_idx, 0], width=1.0, color='#8C1515')
#     axes[1].bar(range(start_idx, end_idx), predictions[start_idx:end_idx, 1], bottom=predictions[start_idx:end_idx, 0], color='#D2C295', label='Probability of 1')
    axes[1].set_xlabel('Position')
    axes[1].set_ylabel('Probability of 0')
#     axes[1].legend()
    axes[1].margins(x=0)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    if save_name is not None:
        plt.savefig(results_path + save_name)
        
    plt.show()

## Load data

In [4]:
with gzip.open(coverage_file_path, 'rt') as coverage_file:
    # Read the file into a pandas DataFrame
    coverage_df = pd.read_csv(coverage_file, sep='\t',
                             usecols=['locus', 'over_10', 'over_20'])

# Expand locus column
coverage_df[['chr', 'pos']] = coverage_df['locus'].str.split(':', expand=True)
coverage_df['pos'] = coverage_df['pos'].astype(int)

# Drop the original locus column
coverage_df = coverage_df.drop('locus', axis=1)

coverage_df

,over_10,over_20,chr,pos
0,0.00000,0.00000,chr1,11819
1,0.00000,0.00000,chr1,11820
2,0.00000,0.00000,chr1,11821
3,0.00000,0.00000,chr1,11822
4,0.00000,0.00000,chr1,11823
...,...,...,...,...
170202922,0.29592,0.23756,chrM,16069
170202923,0.29582,0.23788,chrM,16070
170202924,0.29689,0.23808,chrM,16071
170202925,0.29674,0.23815,chrM,16072


In [5]:
# Read the GTF file into a pandas DataFrame
gene_df = pd.read_csv(gene_annotation_file_path, sep='\t', comment='#', header=None, 
                      names=['chr', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'], 
                      dtype={'start': int, 'end': int})

# Extract 'gene_id' from attributes
gene_df['gene_id'] = gene_df['attribute'].str.extract(r'gene_id "(.*?)"')

# Extract 'gene_type' from attributes
gene_df['gene_type'] = gene_df['attribute'].str.extract(r'gene_type "(.*?)"')

# Extract 'gene_name' from attributes
gene_df['gene_name'] = gene_df['attribute'].str.extract(r'gene_name "(.*?)"')

# Extract 'transcript_id' from attributes
gene_df['transcript_id'] = gene_df['attribute'].str.extract(r'transcript_id "(.*?)"')

# Extract 'transcript' and 'num' from transcript_id
gene_df[['transcript', 'transcript_num']] = gene_df['transcript_id'].str.split('.', expand=True)

# Extract 'transcript_name' from attributes
gene_df['transcript_name'] = gene_df['attribute'].str.extract(r'transcript_name "(.*?)"')

# Drop the original attribute column
gene_df = gene_df.drop('attribute', axis=1)

# Filter rows for protein-coding regions
gene_df = gene_df[(gene_df['gene_type'] == 'protein_coding') & (gene_df['feature'] == 'CDS')]

gene_df

,chr,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,transcript_id,transcript,transcript_num,transcript_name
60,chr1,HAVANA,CDS,65565,65573,.,+,0,ENSG00000186092.7,protein_coding,OR4F5,ENST00000641515.2,ENST00000641515,2,OR4F5-201
63,chr1,HAVANA,CDS,69037,70005,.,+,0,ENSG00000186092.7,protein_coding,OR4F5,ENST00000641515.2,ENST00000641515,2,OR4F5-201
236,chr1,HAVANA,CDS,450743,451678,.,-,0,ENSG00000284733.2,protein_coding,OR4F29,ENST00000426406.4,ENST00000426406,4,OR4F29-201
304,chr1,HAVANA,CDS,685719,686654,.,-,0,ENSG00000284662.2,protein_coding,OR4F16,ENST00000332831.5,ENST00000332831,5,OR4F16-201
524,chr1,HAVANA,CDS,924432,924948,.,+,0,ENSG00000187634.13,protein_coding,SAMD11,ENST00000616016.5,ENST00000616016,5,SAMD11-209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998488,chrM,ENSEMBL,CDS,10470,10763,.,+,0,ENSG00000212907.2,protein_coding,MT-ND4L,ENST00000361335.1,ENST00000361335,1,MT-ND4L-201
1998495,chrM,ENSEMBL,CDS,10760,12137,.,+,0,ENSG00000198886.2,protein_coding,MT-ND4,ENST00000361381.2,ENST00000361381,2,MT-ND4-201
1998509,chrM,ENSEMBL,CDS,12337,14145,.,+,0,ENSG00000198786.2,protein_coding,MT-ND5,ENST00000361567.2,ENST00000361567,2,MT-ND5-201
1998515,chrM,ENSEMBL,CDS,14149,14673,.,-,0,ENSG00000198695.2,protein_coding,MT-ND6,ENST00000361681.2,ENST00000361681,2,MT-ND6-201


In [6]:
# Initialize an empty dataframe to store the concatenated data
variants_df = pd.DataFrame()

# Check if the joined file already exists
if not os.path.exists(variants_file_path):
    # Loop through chromosomes and append dataframes with tqdm
    for filenum in tqdm(range(1, 9), desc='Processing Files'):
        # Read the variants file into a pandas DataFrame
        file_variants_df = pd.read_csv(
            data_path + "output_prefix_0" + str(filenum) + ".tsv.gz",
            sep='-',
            skiprows=1,
            usecols=[0,1],
            names=['chr', 'pos'],
            dtype={'chr': str, 'pos': int},
        )
        
        file_variants_df.drop_duplicates(inplace=True)

        # Append the dataframe to the main dataframe
        variants_df = pd.concat([variants_df, file_variants_df], ignore_index=True)
    
    # Add chr prefix for consistency
    variants_df['chr'] = 'chr' + variants_df['chr'].astype(str)
    
    # Write variants_df to a csv to avoid recomputing
    variants_df.to_csv(variants_file_path, index=False, compression='gzip', sep='\t')
    
else:
    # Read the file into a pandas DataFrame
    variants_df = pd.read_csv(variants_file_path, sep='\t')

# variants_df = variants_df[variants_df['chr'] == 'chr1']
    
variants_df

,chr,pos
0,chr1,10001
1,chr1,10073
2,chr1,10108
3,chr1,10110
4,chr1,10113
...,...,...
934655699,chrY,56887741
934655700,chrY,56887744
934655701,chrY,56887793
934655702,chrY,56887802


## Train HMM on AoU chromosome 2 and get predictions for full genome

In [ ]:
# Define coverage filter
coverage_filter = '0.5_over10' # coverage['over10'] > 0.5
# coverage_filter = '0.9_over20' # coverage['over20'] > 0.9

# Define pattern for coverage file paths
# coverage_file_prefix = data_path + 'gnomad.genomes.r3.0.1.coverage_0.5_over10_chr'
coverage_file_prefix = data_path + 'gnomad_r3_coverage/' + 'gnomad.genomes.r3.0.1.coverage_' + coverage_filter + '_chr'
coverage_file_suffix = '.summary.tsv.gz'

# Parameters for analysis
order = 2

# Define model name (and absolute path)
model_name = results_path + 'HMM_aou_' + coverage_filter + '_chr2'

# Define path to trained HMM
hmm_file_path = model_name + '_model.joblib'

if os.path.exists(hmm_file_path):
    # Load the pretrained model
    model = joblib.load(hmm_file_path)
    
else:
    # Filter rows for chromosome 2
    chr_variants_df = aou_variants_df[aou_variants_df['chr'] == 'chr2']
    chr_coverage_df = pd.read_csv(coverage_file_prefix + '2' + coverage_file_suffix, sep='\t', names=['pos'])

    # Get training data for the chromosome
    observations, positions = get_sequence(chr_coverage_df, chr_variants_df)

    # Fit HMM to Chromosome 2
    model = fit_HMM(observations, order=order)

    # Save the HMM model to a file to avoid retraining
    joblib.dump(model, hmm_file_path)

In [ ]:
# for chromnum in range(1,23):
for chromnum in [1,]:
    print('-'*100)
    print("PROCESSING CHROMOSOME", str(chromnum))

    # Filter rows for given chromosome
    chr_variants_df = aou_variants_df[aou_variants_df['chr'] == 'chr' + str(chromnum)]
    chr_coverage_df = pd.read_csv(coverage_file_prefix + str(chromnum) + coverage_file_suffix, sep='\t', names=['pos'])
    
    # Get training data for the chromosome
    observations, positions = get_sequence(chr_coverage_df, chr_variants_df)

    # Fit HMM and retrieve probabilites
    probabilities = get_HMM_predictions(observations, model, order=order)
    
    # Create a DataFrame with 'pos' reflecting the index of the original sequence and 'prob_0/1' as the predictions
    chr_predictions_df = pd.DataFrame({'chr': 'chr' + str(chromnum),
                                       'pos': positions[0:-order],
                                       'prob_0': probabilities[:, 0], 
                                       'prob_1': probabilities[:, 1], 
                                       'observation': observations[0:-order]
                                      })
    
    predictions_file_prefix = model_name + "_predictions_aou_chr"
    
    # Checkpoint chr_predictions_df to a csv to avoid memory bottleneck
    chr_predictions_df.to_csv(predictions_file_prefix + str(chromnum) + ".tsv.gz", index=False, compression='gzip', sep='\t')
    
    # Plot histograms for predicted vs observed variants
    plot_hist_from_predictions(chr_predictions_df)
    
    # Run regression
    f_statistic, p_value_f_statistic = ols_regression(chr_predictions_df)
    
chr_predictions_df

## Get predictions for AoU WES from the HMM trained on RGC chromosome 2

In [8]:
# Parameters for analysis
# Order of the HMM (defines how many previous states influence the current state)
order = 2

# Coverage threshold for filtering regions with sufficient sequencing depth
coverage_thr = 0.9

# Coverage category to use for filtering (e.g., 'over_20' means regions with coverage > 20)
coverage_category = 'over_20'

# Define the coverage filter based on the selected threshold and category
# Example: '0.9_over20' indicates we are interested in regions where over 90% of positions have coverage > 20
coverage_df = coverage_df[coverage_df[coverage_category] > coverage_thr]

# Define the model name based on the selected coverage filter and chromosome 2
model_name = results_path + 'HMM_rgc_0.9_over20_chr2'

# Load the pretrained model
model = joblib.load(model_name + '_model.joblib')

# Initialize an empty DataFrame to store predictions for all chromosomes
predictions_df = pd.DataFrame(columns=['chr', 'pos','prob_0', 'prob_1', 'observation'])

# Loop over each autosome (chromosomes 1-22) using trange for a progress bar
for chromnum in trange(1, 23, desc="Processing Chromosomes"):
    # Extract the sequence of observations and positions for the current chromosome
    observations, positions = get_sequence(chromnum)

    # Use the trained HMM to predict probabilities for each state (0 and 1) in the sequence
    probabilities = get_HMM_predictions(observations, model, order=order)
    
    # Create a DataFrame with the predictions for the current chromosome
    chr_predictions_df = pd.DataFrame({'chr': 'chr' + str(chromnum),
                                       'pos': positions[0:-order],
                                       'prob_0': probabilities[:, 0], 
                                       'prob_1': probabilities[:, 1], 
                                       'observation': observations[0:-order]
                                      })
    
    # Plot histograms comparing predicted vs observed variants
#     plot_hist_from_predictions(chr_predictions_df)
    
    # Perform regression analysis on the predictions
#     f_statistic, p_value_f_statistic = ols_regression(chr_predictions_df)
    
    # Append the predictions for the current chromosome to the overall DataFrame
    predictions_df = pd.concat([predictions_df, chr_predictions_df], ignore_index=True)

# Save the complete predictions DataFrame to a compressed CSV file to avoid recomputing
predictions_df.to_csv(model_name + "_predictions_aou_wes.tsv.gz", index=False, compression='gzip', sep='\t')

# Display the predictions DataFrame
predictions_df

Processing Chromosomes: 100%|████████████████████████████████████████████████| 22/22 [25:51<00:00, 70.53s/it]


,chr,pos,prob_0,prob_1,observation
0,chr1,925922,9.596546e-31,1.000000,0.0
1,chr1,925923,7.247857e-01,0.275214,0.0
2,chr1,925924,9.135695e-01,0.086431,0.0
3,chr1,925925,9.627417e-01,0.037258,0.0
4,chr1,925926,9.755495e-01,0.024450,0.0
...,...,...,...,...,...
28933587,chr22,50777975,9.796181e-01,0.020382,0.0
28933588,chr22,50777976,9.783619e-01,0.021638,0.0
28933589,chr22,50777977,9.735393e-01,0.026461,0.0
28933590,chr22,50777978,9.550240e-01,0.044976,0.0
